# Processamento de Linguagem Natural (PLN) - Básico

SERPRO - SUPSD - DIVISÃO DE DESENVOLVIMENTO E SUSTENTAÇÃO DE PRODUTOS COGNITIVOS


## Parte 3: Exemplo - Classificação de Sentenças com LSTM

Redes Neurais Recorrentes (RNNs - Recurrent Neural Networks) são uma poderosa família de redes neurais que são amplamente utilizadas para tarefas de modelagem de sequência (por exemplo, previsão de preço de ações, modelagem de linguagem, séries temporais, etc...). A capacidade das RNNs de explorar dependências temporais de entidades em uma sequência as torna poderosas. Neste exercício, utilizaremos uma LSTM, que é uma RNN com células de memória para classificação de sentenças.

Na classificação da sentenças, uma dada sentença deve ser classificada em uma classe (categoria). Usaremos um banco de dados de perguntas, onde cada pergunta é rotulada pelo assunto da pergunta. Por exemplo, a pergunta "Quem foi Abraham Lincoln?" será uma pergunta e seu rótulo será Pessoa. Para isso, usaremos um conjunto de dados de classificação de frases disponível em http://cogcomp.org/Data/QA/QC/; aqui você encontrará sentenças de treinamento e seus respectivos rótulos e 500 sentenças de teste.

Categorias: http://cogcomp.org/Data/QA/QC/definition.html

Usaremos a seguinte estratégia:  
 - Extrair os dados
 - Pré-processamento dos dados
 - Construir a Rede Neural LSTM
 - Treinamento do modelo
 - Avaliar o modelo
 - Realizando Predições com dados de Teste

In [1]:
#Imports
import os
import numpy as np 
import pandas as pd
import re
import operator

import tensorflow as tf

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout,TimeDistributed,Activation
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
tf.__version__

'1.13.1'

## Verificando o Dataset

Usaremos o __[dataset](http://cogcomp.cs.illinois.edu/Data/QA/QC/)__ que é composto de perguntas como entradas e seu respectivo tipo como saída. Por exemplo: quem era Abraham Lincon? E a saída ou rótulo seria Humano.


In [3]:
# Diretório
dir_name = 'data/question-classif-data'

# Dataset de treino
#train_filename = 'train_1000.label'
train_filename = 'train_5500.label'

# Dataset de teste
test_filename = 'TREC_10.label'

In [4]:
# Verifica se os arquivos existem
filenames = [train_filename,test_filename]
num_files = len(filenames)
for i in range(len(filenames)):
    file_exists = os.path.isfile(os.path.join(dir_name,filenames[i]))
    assert file_exists
print('Arquivos encontrados e verificados!')

Arquivos encontrados e verificados!


## Carregando e visualizando os dados

Abaixo nós carregamos o texto e visualizar alguns dados

In [5]:
# Função para leitura dos dados
def read_data_into_pandas(filename):
    col_names =  ['Category', 'Subcategory', 'Question']
    data  = pd.DataFrame(columns = col_names)
    with open(filename,'r',encoding='latin-1') as f: 
        count=1;
        for row in f:
            #Quebra a linha em duas no primeiro espaço (category:subcategory and question)
            row_str = row.split(" ",1)
            #Separa a categoria da subcategoria
            row_category = row_str[0].split(":")            
            data.loc[len(data)] = [row_category[0],row_category[1],row_str[1].lower()]
            count+=1
    return data
# Carrega os dados em DataFrames específicos para treinamento e teste
data_train = read_data_into_pandas(os.path.join(dir_name,train_filename))
data_test = read_data_into_pandas(os.path.join(dir_name,test_filename))

# Concatena os dados de treinamento e teste. Será utilizado para construir um vocabulário único
data = pd.concat([data_train,data_test])
print(data.head())

  Category Subcategory                                           Question
0     DESC      manner  how did serfdom develop in and then leave russ...
1     ENTY      cremat  what films featured the character popeye doyle...
2     DESC      manner  how can i find a list of celebrities ' real na...
3     ENTY      animal  what fowl grabs the spotlight after the chines...
4     ABBR         exp                  what is the full form of .com ?\n


In [6]:
# Visualizando algumas informações sobre os dados
# Exemplo Treinamento
print('Treinamento')
print(' - Número questões na categoria HUMAN : {}'.format(data_train[data_train['Category'] == 'HUM'].size))
print(' - Exemplos de questões na categoria HUMAN :')
print(data_train[data_train['Category'] == 'HUM']['Question'].head())
print('')
# Exemplo Teste
print('Teste')
print(' - Número questões na categoria HUMAN : {}'.format(data_test[ data_test['Category'] == 'HUM'].size))
print(' - Exemplos de questões na categoria HUMAN :')
print(data_test[data_test['Category'] == 'HUM']['Question'].head())

Treinamento
 - Número questões na categoria HUMAN : 3669
 - Exemplos de questões na categoria HUMAN :
5     what contemptible scoundrel stole the cork fro...
6     what team did baseball 's st. louis browns bec...
7                     what is the oldest profession ?\n
9     name the scar-faced bounty hunter of the old w...
12                 who was the pride of the yankees ?\n
Name: Question, dtype: object

Teste
 - Número questões na categoria HUMAN : 195
 - Exemplos de questões na categoria HUMAN :
2                                   who was galileo ?\n
6     george bush purchased a small interest in whic...
11                 what person 's head is on a dime ?\n
13    who was the first man to fly across the pacifi...
17      who developed the vaccination against polio ?\n
Name: Question, dtype: object


## Pré-processamento e Vetorização dos Dados para o Treinamento

Utilizaremos a classe Tokenizer do __[Keras](https://keras.io/)__ para montar nosso vocabulário. Ela permite vetorizar o texto do corpus, transformando o texto na sequência de inteiros (cada inteiro representa o índice do token em um dicionário) 
https://keras.io/preprocessing/text/

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Question'].values)
print("Tamanho do Vocabulário: ", len(tokenizer.word_index))
print("10 primeiras palavras do vocabulário: ")
iterator = iter(tokenizer.word_index.items())
for i in range(10):
    print(next(iterator))

Tamanho do Vocabulário:  8761
10 primeiras palavras do vocabulário: 
('the', 1)
('what', 2)
('is', 3)
('of', 4)
('in', 5)
('a', 6)
('how', 7)
("'s", 8)
('was', 9)
('who', 10)


In [8]:
# Vetorização dos dados de treinamento
train_X = tokenizer.texts_to_sequences(data_train['Question'].values)
print("Exemplo de frase vetorizada:", train_X[0])
print("Exemplo de frase vetorizada:", train_X[1])
print("Exemplo de frase vetorizada:", train_X[3])

# Padding dos dados de treinamento: preencher com zeros para que todas as frases fiquem do mesmo tamanho
# Necessário para alimentar a rede neural  
train_X = pad_sequences(train_X,value=0.)

print("Exemplo de frase vetorizada preenchida com zeros: ", train_X[0])
print("Shape do vetor de treinamento:",train_X.shape)
print("Tamanho da maior frase:",train_X.shape[1])

# Vetorização dos labels para os dados de treinamento
train_labels = pd.get_dummies(data_train['Category'])
print("Categorias: ",train_labels.columns)
train_Y = pd.get_dummies(data_train['Category']).values
print("Exemplo vetor de label:",train_Y[0])


Exemplo de frase vetorizada: [7, 15, 3656, 2318, 5, 14, 440, 852, 1033]
Exemplo de frase vetorizada: [2, 853, 639, 1, 134, 1319, 3657]
Exemplo de frase vetorizada: [2, 3658, 3659, 1, 3660, 137, 1, 499, 46, 4, 1, 1683]
Exemplo de frase vetorizada preenchida com zeros:  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    7   15 3656 2318
    5   14  440  852 1033]
Shape do vetor de treinamento: (5452, 33)
Tamanho da maior frase: 33
Categorias:  Index(['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'], dtype='object')
Exemplo vetor de label: [0 1 0 0 0 0]


In [9]:
# Vetorização dos dados de teste
test_X = tokenizer.texts_to_sequences(data_test['Question'].values)

# Temos que usar o mesmo tamanho de frases pros dados de testes, por isso usamos o tamanho dos dados de treinamento
test_X = pad_sequences(test_X,maxlen=train_X.shape[1],value=0.)
print("Shape do vetor de teste",test_X.shape)
test_labels = pd.get_dummies(data_test['Category']).columns
test_Y = pd.get_dummies(data_test['Category']).values

Shape do vetor de teste (500, 33)


## Construção da Rede Neural (LSTM)
http://tflearn.org/layers/recurrent/#lstm

https://keras.io/getting-started/sequential-model-guide/#sequence-classification-with-lstm

Vamos criar uma rede LSTM com a seguinte arquitetura:
- O input será uma matriz com dimensão X (onde x é maior número de palavras de um documento)
- Camada de entrada terá o número de neurônio igual ao número de features (words embeddings) 
- Uma camada LSTM com 128 unidades nas camadas ocultas nas células LSTM. Geralmente quanto maior melhor é desempenho. Os valores comuns são 128, 256, 512, etc.
- Camada Dropout para evitar o overfitting
- E uma camada totalmente conectada com 6 neurônios, referente as quantidade de categorias, função de ativação Softmax com as probabilidades para cada classe
- Por fim uma camada de regressão com a função de perda e otimização

In [10]:
keras.backend.clear_session()

#Size of the vocabulary, i.e. maximum integer index + 1
max_features=len(tokenizer.word_index)+1

#Construção da Rede
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=256, input_length=train_X.shape[1]))
model.add(LSTM(units=128))
model.add(Dropout(0.5))
model.add(Dense(6,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 33, 256)           2243072   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
Total params: 2,440,966
Trainable params: 2,440,966
Non-trainable params: 0
_________________________________________________________________
None


## Treinamento e Avaliação do Modelo

Para treinar o modelo vamos utilizar 90% dos dados para treinamento e 10% para validação. 

Lembrado que os hiperparâmetros do treinamento podem ser ajustados de acordo com a necessidade e com o intuito de melhorar a acurácia.

Vamos criar um callback para visualizar os dados do treinamento no tesorboard

In [12]:
from keras.callbacks import TensorBoard
import datetime
# Ativando o Tensorboard para poder monitorar com o comando abaixo em uym outro terminal:
# > tensorboard --logdir=logskeras/
tensorboard = TensorBoard(log_dir='logskeras/{}'.format(datetime.datetime.today().strftime('%Y%m%d_%H%M')))

model.fit(x=train_X, y=train_Y, batch_size=32, epochs=5, verbose=1, validation_split=0.1,callbacks=[tensorboard])

Instructions for updating:
Use tf.cast instead.
Train on 4906 samples, validate on 546 samples
Epoch 1/5
4906/4906 [==============================] - 5s 1ms/step - loss: 1.3478 - acc: 0.4609 - val_loss: 0.8664 - val_acc: 0.6868
Epoch 2/5
4906/4906 [==============================] - 5s 947us/step - loss: 0.6183 - acc: 0.7945 - val_loss: 0.6462 - val_acc: 0.7967
Epoch 3/5
4906/4906 [==============================] - 5s 939us/step - loss: 0.3281 - acc: 0.8997 - val_loss: 0.5528 - val_acc: 0.8352
Epoch 4/5
4906/4906 [==============================] - 5s 919us/step - loss: 0.1973 - acc: 0.9452 - val_loss: 0.7781 - val_acc: 0.7894
Epoch 5/5
4906/4906 [==============================] - 5s 931us/step - loss: 0.1178 - acc: 0.9662 - val_loss: 0.5727 - val_acc: 0.8205


Avaliando o modelo com os dados de teste.

In [13]:
loss,acc = model.evaluate(test_X, test_Y, verbose = 2)
print("Acurácia: %.2f" % (acc))

Acurácia: 0.87


## Realizando Predições com dados de Teste

Vamos visualizar algumas predições dos dados de teste

In [20]:
# Predição dos dados de testes
predictions = model.predict(test_X)

# Criação do dicionário reverso (key -> Índice ; value -> Token)
reverse_dictionary = dict(zip(tokenizer.word_index.values(), tokenizer.word_index.keys()))

#print(reverse_dictionary)

# Varre os dados de teste e imprime a predição
for i in range(len(test_X[:10])):
    frase='';
    for j in test_X[i]:
        if j != 0:
            frase+=reverse_dictionary[j]+' '
    print('Frase ->',frase)
    print('    Label category->',data_test['Category'][i])
    print('    Label predict->', test_labels[np.argmax(predictions[i])])
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

Frase -> how far is it from denver to aspen 
    Label category-> NUM
    Label predict-> NUM
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> what county is modesto california in 
    Label category-> LOC
    Label predict-> LOC
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> who was galileo 
    Label category-> HUM
    Label predict-> HUM
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> what is an atom 
    Label category-> DESC
    Label predict-> DESC
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> when did hawaii become a state 
    Label category-> NUM
    Label predict-> NUM
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> how tall is the sears building 
    Label category-> NUM
    Label predict-> NUM
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Frase -> george bush purchased a small interest in which baseball team 
    Label category-> HUM
    Label predict-> E

Realizando predições com um dado qualquer

In [15]:
test = tokenizer.texts_to_sequences(["where is Brazil?"])
test = pad_sequences(test,maxlen=train_X.shape[1],value=0.)

predictions = model.predict(test)
with np.printoptions(precision=3, suppress=True):
    print(predictions)
    print('Labels ->',test_labels)
    print('Label predict->', test_labels[np.argmax(predictions[0])])   

[[0.002 0.002 0.003 0.002 0.982 0.01 ]]
Labels -> Index(['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM'], dtype='object')
Label predict-> LOC


## Empilhamento de camadas LSTM para classificação

É possível fazer o empilhamento de camadas LSTM , para isso é necessário ativar o parâmetro __return_sequences__. Ele informa se a camada LSTM irá retornar apenas o última saída da sequência ou a sequência inteira (caso true). 

Para mais informações acessar o Sequencial Guide Mode(https://keras.io/getting-started/sequential-model-guide/)

In [16]:
#help
?LSTM

In [17]:
keras.backend.clear_session()

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=256, input_length=train_X.shape[1]))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=True)) 
model.add(LSTM(128))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 33, 256)           2243072   
_________________________________________________________________
lstm_1 (LSTM)                (None, 33, 128)           197120    
_________________________________________________________________
lstm_2 (LSTM)                (None, 33, 128)           131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
Total params: 2,704,134
Trainable params: 2,704,134
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
model.fit(train_X, train_Y, batch_size=32, epochs=5, validation_split=0.1)

Train on 4906 samples, validate on 546 samples
Epoch 1/5
4906/4906 [==============================] - 10s 2ms/step - loss: 1.2757 - acc: 0.4670 - val_loss: 1.0806 - val_acc: 0.5769
Epoch 2/5
4906/4906 [==============================] - 9s 2ms/step - loss: 0.6695 - acc: 0.7693 - val_loss: 0.6968 - val_acc: 0.7784
Epoch 3/5
4906/4906 [==============================] - 9s 2ms/step - loss: 0.3917 - acc: 0.8732 - val_loss: 0.6750 - val_acc: 0.7894
Epoch 4/5
4906/4906 [==============================] - 9s 2ms/step - loss: 0.2405 - acc: 0.9266 - val_loss: 0.6690 - val_acc: 0.8059
Epoch 5/5
4906/4906 [==============================] - 9s 2ms/step - loss: 0.1583 - acc: 0.9515 - val_loss: 0.7159 - val_acc: 0.8278


In [19]:
loss,acc = model.evaluate(test_X, test_Y, verbose = 2)
print("Acurácia: %.2f" % (acc))

Acurácia: 0.85


## Exercícios

---

### Exercício 1

Modifique este jupyter para que o modelo obtenha um melhor desempenho.

Primeiro crie uma função de pré-processamento dos dados para realizar as seguintes tarefas:

- Remoção de Pontuação
- Tokenização
- Remoção de Stop words
- Aplicação de Stemmer

Dica: Essa função foi criada e utilizada no jupyter anterior.

Reprocesse e veja os resultados.

---

---

### Exercício 2

Faça modificando para tentar melhorar a acurácia do modelo:

- Uso de Lemmatization ao invés de Stemming
- Modificações dos hiperparâmetros da rede neural, tais como 'n_epoch' e 'batch_size'
- Modifique a arquitetura da rede neural, tal como a quantidade de unidades da célula LSTM

Qual configuração obteve melhores resultados?

---

## Fim